## Modelling

This note book explains the different steps in the machine learning model for the binary classfication model. First the model is trained on the full dataset to obtain the optimal features followed by hyper parameter tunning and model performance estimate using Nested Cross Validation.

* Nested Cross Validation for
    * Feature selection 
    * hyper parameter tunning 
* Performance metrics
* Baseline Models

### Binary Classification
At the end of this section we will obtain  the optimal Binary Classification models and the performance estimates, 
for a 10% threshold. Two models are implemented: Random Forest Classifier, XGBoost Classifier. 
First, the model is trained on the full dataset to obtain the optimal features followed by a model 
that obtains the performance estimate using Nested Cross Validation.


In [41]:

 from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
import importlib

from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    StratifiedKFold,
    KFold,
)

from sklearn.feature_selection import (
    SelectKBest,
    RFECV,
    SequentialFeatureSelector,
    RFE,
    mutual_info_regression,
    f_regression,
    mutual_info_classif,
)

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

from sklearn.metrics import (
    recall_score,
    mean_absolute_error,
    mean_squared_error,
    f1_score,
    precision_score,
    confusion_matrix,
    make_scorer,
)

from sklearn.inspection import permutation_importance
from sklearn.linear_model import LinearRegression

import numpy as np
from numpy.lib.function_base import average
import pandas as pd
import matplotlib.pyplot as plt

import os
import random
import pickle
import openpyxl
import pickle
import geopandas as gpd


from xgboost import XGBClassifier
from xgboost.sklearn import XGBRegressor
import eli5
from eli5.sklearn import PermutationImportance
import xgboost as xgb

In [42]:

def binary_damage_class(x):
    damage = x[0]   
    if damage > 10:
        value = 1
    else:
        value = 0
    return value


def splitting_train_test(df):

    # To save the train and test sets
    df_train_list = []
    df_test_list = []

    # List of typhoons that are to be used as a test set 
 
    typhoons_with_impact_data=list(np.unique(df.typhoon))

    for typhoon in typhoons_with_impact_data:
        if len(df[df["typhoon"] == typhoon]) >1:
            df_train_list.append(df[df["typhoon"] != typhoon])
            df_test_list.append(df[df["typhoon"] == typhoon])

    return df_train_list, df_test_list

def unweighted_random(y_train, y_test):
    options = y_train.value_counts(normalize=True)
    y_pred = random.choices(population=list(options.index), k=len(y_test))
    return y_pred

def weighted_random(y_train, y_test):
    options = y_train.value_counts()
    y_pred = random.choices(
        population=list(options.index), weights=list(options.values), k=len(y_test)
    )
    return y_pred

In [43]:
# Setting directory

wor_dir="/home/fbf/"
wor_dir='C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/IBF-Typhoon-model/'

os.chdir(wor_dir)

cdir = os.getcwd()

In [44]:
# Import functions
from models.binary_classification.rf_binary import (rf_binary_features,rf_binary_performance,)
from models.binary_classification.xgb_binary import (xgb_binary_features,xgb_binary_performance,)
from models.regression.rf_regression import (rf_regression_features,rf_regression_performance,)
from models.regression.xgb_regression import (xgb_regression_features,xgb_regression_performance,)




In [45]:


combined_input_data=pd.read_csv("data/model_input/combined_input_data.csv")

combined_input_data["DAM_binary_dmg"] = combined_input_data[["DAM_perc_dmg"]].apply(binary_damage_class, axis="columns")


combined_input_data =combined_input_data.filter(['typhoon','HAZ_rainfall_Total', 
        'HAZ_rainfall_max_6h',
        'HAZ_rainfall_max_24h',
        'HAZ_v_max',
        'HAZ_dis_track_min',
        'GEN_landslide_per',
        'GEN_stormsurge_per',
        'GEN_Bu_p_inSSA', 
        'GEN_Bu_p_LS', 
        'GEN_Red_per_LSbldg',
        'GEN_Or_per_LSblg', 
        'GEN_Yel_per_LSSAb', 
        'GEN_RED_per_SSAbldg',
        'GEN_OR_per_SSAbldg',
        'GEN_Yellow_per_LSbl',
        'TOP_mean_slope',
        'TOP_mean_elevation_m', 
        'TOP_ruggedness_stdev', 
        'TOP_mean_ruggedness',
        'TOP_slope_stdev', 
        'VUL_poverty_perc',
        'GEN_with_coast',
        'GEN_coast_length', 
        'VUL_Housing_Units',
        'VUL_StrongRoof_StrongWall', 
        'VUL_StrongRoof_LightWall',
        'VUL_StrongRoof_SalvageWall', 
        'VUL_LightRoof_StrongWall',
        'VUL_LightRoof_LightWall', 
        'VUL_LightRoof_SalvageWall',
        'VUL_SalvagedRoof_StrongWall',
        'VUL_SalvagedRoof_LightWall',
        'VUL_SalvagedRoof_SalvageWall', 
        'VUL_vulnerable_groups',
        'VUL_pantawid_pamilya_beneficiary',
        'DAM_binary_dmg'])


In [46]:
features =['HAZ_rainfall_Total', 
        'HAZ_rainfall_max_6h',
        'HAZ_rainfall_max_24h',
        'HAZ_v_max',
        'HAZ_dis_track_min',
        'GEN_landslide_per',
        'GEN_stormsurge_per',
        'GEN_Bu_p_inSSA', 
        'GEN_Bu_p_LS', 
        'GEN_Red_per_LSbldg',
        'GEN_Or_per_LSblg', 
        'GEN_Yel_per_LSSAb', 
        'GEN_RED_per_SSAbldg',
        'GEN_OR_per_SSAbldg',
        'GEN_Yellow_per_LSbl',
        'TOP_mean_slope',
        'TOP_mean_elevation_m', 
        'TOP_ruggedness_stdev', 
        'TOP_mean_ruggedness',
        'TOP_slope_stdev', 
        'VUL_poverty_perc',
        'GEN_with_coast',
        'GEN_coast_length', 
        'VUL_Housing_Units',
        'VUL_StrongRoof_StrongWall', 
        'VUL_StrongRoof_LightWall',
        'VUL_StrongRoof_SalvageWall', 
        'VUL_LightRoof_StrongWall',
        'VUL_LightRoof_LightWall', 
        'VUL_LightRoof_SalvageWall',
        'VUL_SalvagedRoof_StrongWall',
        'VUL_SalvagedRoof_LightWall',
        'VUL_SalvagedRoof_SalvageWall', 
        'VUL_vulnerable_groups',
        'VUL_pantawid_pamilya_beneficiary']

####  Random forest 

In [19]:
combined_input_data.query("DAM_binary_dmg>0")

,typhoon,HAZ_rainfall_Total,HAZ_rainfall_max_6h,HAZ_rainfall_max_24h,HAZ_v_max,HAZ_dis_track_min,GEN_landslide_per,GEN_stormsurge_per,GEN_Bu_p_inSSA,GEN_Bu_p_LS,...,VUL_StrongRoof_SalvageWall,VUL_LightRoof_StrongWall,VUL_LightRoof_LightWall,VUL_LightRoof_SalvageWall,VUL_SalvagedRoof_StrongWall,VUL_SalvagedRoof_LightWall,VUL_SalvagedRoof_SalvageWall,VUL_vulnerable_groups,VUL_pantawid_pamilya_beneficiary,DAM_binary_dmg
39,durian2006,128.785714,7.623810,4.991964,67.297247,16.034037,0.61,0.00,0.00,0.61,...,0.384329,0.384329,0.384329,0.384329,0.384329,0.384329,0.384329,2.458422,28.331475,1
40,durian2006,137.600000,10.846970,5.338258,54.729980,14.053818,2.58,7.27,7.27,2.58,...,0.705429,0.705429,0.705429,0.705429,0.705429,0.705429,0.705429,2.364638,40.058196,1
67,durian2006,236.800000,23.022917,9.250000,68.469676,28.294582,5.76,1.26,1.26,5.76,...,1.826588,1.826588,1.826588,1.826588,1.826588,1.826588,1.826588,3.152242,31.665475,1
79,durian2006,166.710000,11.723333,6.353750,64.000391,20.452986,0.97,0.03,0.03,0.97,...,0.435443,0.435443,0.435443,0.435443,0.435443,0.435443,0.435443,2.276694,30.884774,1
80,durian2006,218.475000,20.925000,8.506771,76.858109,16.963048,6.14,2.50,2.50,6.14,...,1.228233,1.228233,1.228233,1.228233,1.228233,1.228233,1.228233,1.762301,22.647775,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23423,vongfong2020,254.525000,23.410417,9.786458,47.579864,0.993916,0.00,0.00,0.00,0.00,...,0.742781,0.742781,0.742781,0.742781,0.742781,0.742781,0.742781,2.409639,44.973852,1
23461,vongfong2020,255.687500,21.612500,9.660417,48.218915,7.514910,0.00,0.00,0.00,0.00,...,1.108188,1.108188,1.108188,1.108188,1.108188,1.108188,1.108188,7.043011,27.350081,1
23877,vongfong2020,258.160000,18.825000,9.100833,49.041771,0.746003,0.00,0.00,0.00,0.00,...,1.050984,1.050984,1.050984,1.050984,1.050984,1.050984,1.050984,4.078437,34.293595,1
23958,vongfong2020,212.800000,15.002381,7.574107,43.430665,1.708735,0.00,0.00,0.00,0.00,...,0.546755,0.546755,0.546755,0.546755,0.546755,0.546755,0.546755,5.823910,39.559633,1


In [30]:

df=combined_input_data.dropna()
 
#combined_input_data = combined_input_data[combined_input_data['DAM_perc_dmg'].notnull()]
X = df[features]
y = df["DAM_binary_dmg"]

# Setting the train and the test sets for obtaining performance estimate
df_train_list, df_test_list = splitting_train_test(df)

In [31]:
# Setting the random forest search grid
rf_search_space = [
    {
        "estimator__n_estimators": [100, 250],
        "estimator__max_depth": [20, None],
        "estimator__min_samples_split": [2, 8, 10, 15],
        "estimator__min_samples_leaf": [1, 3, 5],
    }
]

# Obtaining the selected features based on the full dataset
selected_features_rf_binary, selected_params_rf_binary_full = rf_binary_features(
    X=X,
    y=y,
    features=features,
    search_space=rf_search_space,
    cv_splits=5,
    class_weight="balanced",
    min_features_to_select=1,
    GS_score="f1",
    GS_randomized=False,
    GS_n_iter=10,
    verbose=10,
)

print(f"Number of selected features RF Binary: {len(selected_features_rf_binary)}")
print(f"Selected features RF Binary: {selected_features_rf_binary}")
print(f"Selected Parameters RF Binary {selected_params_rf_binary_full}")

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV 1/5; 1/48] START estimator__max_depth=20, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=100
[CV 1/5; 1/48] END estimator__max_depth=20, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=100;, score=(train=0.962, test=0.516) total time= 2.9min
[CV 2/5; 1/48] START estimator__max_depth=20, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=100
[CV 2/5; 1/48] END estimator__max_depth=20, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=100;, score=(train=0.918, test=0.611) total time= 2.8min
[CV 3/5; 1/48] START estimator__max_depth=20, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=100
[CV 3/5; 1/48] END estimator__max_depth=20, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=100;, score=(train=0

[CV 1/5; 6/48] END estimator__max_depth=20, estimator__min_samples_leaf=1, estimator__min_samples_split=10, estimator__n_estimators=250;, score=(train=0.853, test=0.574) total time= 6.9min
[CV 2/5; 6/48] START estimator__max_depth=20, estimator__min_samples_leaf=1, estimator__min_samples_split=10, estimator__n_estimators=250
[CV 2/5; 6/48] END estimator__max_depth=20, estimator__min_samples_leaf=1, estimator__min_samples_split=10, estimator__n_estimators=250;, score=(train=0.854, test=0.550) total time= 6.6min
[CV 3/5; 6/48] START estimator__max_depth=20, estimator__min_samples_leaf=1, estimator__min_samples_split=10, estimator__n_estimators=250
[CV 3/5; 6/48] END estimator__max_depth=20, estimator__min_samples_leaf=1, estimator__min_samples_split=10, estimator__n_estimators=250;, score=(train=0.781, test=0.532) total time= 5.9min
[CV 4/5; 6/48] START estimator__max_depth=20, estimator__min_samples_leaf=1, estimator__min_samples_split=10, estimator__n_estimators=250
[CV 4/5; 6/48] END 

[CV 2/5; 11/48] END estimator__max_depth=20, estimator__min_samples_leaf=3, estimator__min_samples_split=8, estimator__n_estimators=100;, score=(train=0.817, test=0.575) total time= 2.2min
[CV 3/5; 11/48] START estimator__max_depth=20, estimator__min_samples_leaf=3, estimator__min_samples_split=8, estimator__n_estimators=100
[CV 3/5; 11/48] END estimator__max_depth=20, estimator__min_samples_leaf=3, estimator__min_samples_split=8, estimator__n_estimators=100;, score=(train=0.864, test=0.565) total time= 2.5min
[CV 4/5; 11/48] START estimator__max_depth=20, estimator__min_samples_leaf=3, estimator__min_samples_split=8, estimator__n_estimators=100
[CV 4/5; 11/48] END estimator__max_depth=20, estimator__min_samples_leaf=3, estimator__min_samples_split=8, estimator__n_estimators=100;, score=(train=0.832, test=0.576) total time= 2.5min
[CV 5/5; 11/48] START estimator__max_depth=20, estimator__min_samples_leaf=3, estimator__min_samples_split=8, estimator__n_estimators=100
[CV 5/5; 11/48] END

[CV 2/5; 16/48] END estimator__max_depth=20, estimator__min_samples_leaf=3, estimator__min_samples_split=15, estimator__n_estimators=250;, score=(train=0.769, test=0.554) total time= 5.8min
[CV 3/5; 16/48] START estimator__max_depth=20, estimator__min_samples_leaf=3, estimator__min_samples_split=15, estimator__n_estimators=250
[CV 3/5; 16/48] END estimator__max_depth=20, estimator__min_samples_leaf=3, estimator__min_samples_split=15, estimator__n_estimators=250;, score=(train=0.780, test=0.536) total time= 5.8min
[CV 4/5; 16/48] START estimator__max_depth=20, estimator__min_samples_leaf=3, estimator__min_samples_split=15, estimator__n_estimators=250
[CV 4/5; 16/48] END estimator__max_depth=20, estimator__min_samples_leaf=3, estimator__min_samples_split=15, estimator__n_estimators=250;, score=(train=0.764, test=0.573) total time= 4.9min
[CV 5/5; 16/48] START estimator__max_depth=20, estimator__min_samples_leaf=3, estimator__min_samples_split=15, estimator__n_estimators=250
[CV 5/5; 16/4

[CV 3/5; 21/48] END estimator__max_depth=20, estimator__min_samples_leaf=5, estimator__min_samples_split=10, estimator__n_estimators=100;, score=(train=0.813, test=0.518) total time= 2.2min
[CV 4/5; 21/48] START estimator__max_depth=20, estimator__min_samples_leaf=5, estimator__min_samples_split=10, estimator__n_estimators=100
[CV 4/5; 21/48] END estimator__max_depth=20, estimator__min_samples_leaf=5, estimator__min_samples_split=10, estimator__n_estimators=100;, score=(train=0.786, test=0.588) total time= 2.2min
[CV 5/5; 21/48] START estimator__max_depth=20, estimator__min_samples_leaf=5, estimator__min_samples_split=10, estimator__n_estimators=100
[CV 5/5; 21/48] END estimator__max_depth=20, estimator__min_samples_leaf=5, estimator__min_samples_split=10, estimator__n_estimators=100;, score=(train=0.807, test=0.520) total time= 2.4min
[CV 1/5; 22/48] START estimator__max_depth=20, estimator__min_samples_leaf=5, estimator__min_samples_split=10, estimator__n_estimators=250
[CV 1/5; 22/4

[CV 3/5; 26/48] END estimator__max_depth=None, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=250;, score=(train=0.981, test=0.448) total time= 6.7min
[CV 4/5; 26/48] START estimator__max_depth=None, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=250
[CV 4/5; 26/48] END estimator__max_depth=None, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=250;, score=(train=0.988, test=0.500) total time= 6.7min
[CV 5/5; 26/48] START estimator__max_depth=None, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=250
[CV 5/5; 26/48] END estimator__max_depth=None, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=250;, score=(train=0.995, test=0.494) total time= 6.5min
[CV 1/5; 27/48] START estimator__max_depth=None, estimator__min_samples_leaf=1, estimator__min_samples_split=8, estimator__n_estimators=100
[CV 1/5

[CV 3/5; 31/48] END estimator__max_depth=None, estimator__min_samples_leaf=1, estimator__min_samples_split=15, estimator__n_estimators=100;, score=(train=0.887, test=0.592) total time= 2.3min
[CV 4/5; 31/48] START estimator__max_depth=None, estimator__min_samples_leaf=1, estimator__min_samples_split=15, estimator__n_estimators=100
[CV 4/5; 31/48] END estimator__max_depth=None, estimator__min_samples_leaf=1, estimator__min_samples_split=15, estimator__n_estimators=100;, score=(train=0.897, test=0.562) total time= 2.1min
[CV 5/5; 31/48] START estimator__max_depth=None, estimator__min_samples_leaf=1, estimator__min_samples_split=15, estimator__n_estimators=100
[CV 5/5; 31/48] END estimator__max_depth=None, estimator__min_samples_leaf=1, estimator__min_samples_split=15, estimator__n_estimators=100;, score=(train=0.880, test=0.495) total time= 2.2min
[CV 1/5; 32/48] START estimator__max_depth=None, estimator__min_samples_leaf=1, estimator__min_samples_split=15, estimator__n_estimators=250
[

[CV 3/5; 36/48] END estimator__max_depth=None, estimator__min_samples_leaf=3, estimator__min_samples_split=8, estimator__n_estimators=250;, score=(train=0.916, test=0.602) total time= 5.2min
[CV 4/5; 36/48] START estimator__max_depth=None, estimator__min_samples_leaf=3, estimator__min_samples_split=8, estimator__n_estimators=250
[CV 4/5; 36/48] END estimator__max_depth=None, estimator__min_samples_leaf=3, estimator__min_samples_split=8, estimator__n_estimators=250;, score=(train=0.910, test=0.600) total time= 6.2min
[CV 5/5; 36/48] START estimator__max_depth=None, estimator__min_samples_leaf=3, estimator__min_samples_split=8, estimator__n_estimators=250
[CV 5/5; 36/48] END estimator__max_depth=None, estimator__min_samples_leaf=3, estimator__min_samples_split=8, estimator__n_estimators=250;, score=(train=0.910, test=0.536) total time= 6.4min
[CV 1/5; 37/48] START estimator__max_depth=None, estimator__min_samples_leaf=3, estimator__min_samples_split=10, estimator__n_estimators=100
[CV 1/

[CV 3/5; 41/48] END estimator__max_depth=None, estimator__min_samples_leaf=5, estimator__min_samples_split=2, estimator__n_estimators=100;, score=(train=0.829, test=0.549) total time= 1.7min
[CV 4/5; 41/48] START estimator__max_depth=None, estimator__min_samples_leaf=5, estimator__min_samples_split=2, estimator__n_estimators=100
[CV 4/5; 41/48] END estimator__max_depth=None, estimator__min_samples_leaf=5, estimator__min_samples_split=2, estimator__n_estimators=100;, score=(train=0.822, test=0.615) total time= 2.4min
[CV 5/5; 41/48] START estimator__max_depth=None, estimator__min_samples_leaf=5, estimator__min_samples_split=2, estimator__n_estimators=100
[CV 5/5; 41/48] END estimator__max_depth=None, estimator__min_samples_leaf=5, estimator__min_samples_split=2, estimator__n_estimators=100;, score=(train=0.838, test=0.544) total time= 2.1min
[CV 1/5; 42/48] START estimator__max_depth=None, estimator__min_samples_leaf=5, estimator__min_samples_split=2, estimator__n_estimators=250
[CV 1/5

[CV 3/5; 46/48] END estimator__max_depth=None, estimator__min_samples_leaf=5, estimator__min_samples_split=10, estimator__n_estimators=250;, score=(train=0.851, test=0.558) total time= 6.2min
[CV 4/5; 46/48] START estimator__max_depth=None, estimator__min_samples_leaf=5, estimator__min_samples_split=10, estimator__n_estimators=250
[CV 4/5; 46/48] END estimator__max_depth=None, estimator__min_samples_leaf=5, estimator__min_samples_split=10, estimator__n_estimators=250;, score=(train=0.837, test=0.615) total time= 5.1min
[CV 5/5; 46/48] START estimator__max_depth=None, estimator__min_samples_leaf=5, estimator__min_samples_split=10, estimator__n_estimators=250
[CV 5/5; 46/48] END estimator__max_depth=None, estimator__min_samples_leaf=5, estimator__min_samples_split=10, estimator__n_estimators=250;, score=(train=0.842, test=0.552) total time= 5.8min
[CV 1/5; 47/48] START estimator__max_depth=None, estimator__min_samples_leaf=5, estimator__min_samples_split=15, estimator__n_estimators=100
[

In [33]:
selected_features_rf_binary

['HAZ_rainfall_Total',
 'HAZ_rainfall_max_6h',
 'HAZ_rainfall_max_24h',
 'HAZ_v_max',
 'HAZ_dis_track_min',
 'GEN_landslide_per',
 'GEN_stormsurge_per',
 'TOP_mean_slope',
 'TOP_mean_elevation_m',
 'TOP_ruggedness_stdev',
 'TOP_mean_ruggedness',
 'TOP_slope_stdev',
 'VUL_poverty_perc',
 'GEN_coast_length',
 'VUL_Housing_Units',
 'VUL_StrongRoof_StrongWall',
 'VUL_StrongRoof_SalvageWall',
 'VUL_LightRoof_StrongWall',
 'VUL_LightRoof_SalvageWall',
 'VUL_SalvagedRoof_StrongWall',
 'VUL_vulnerable_groups',
 'VUL_pantawid_pamilya_beneficiary']

In [35]:
selected_features_rf_binary=[
    'HAZ_rainfall_Total',
     'HAZ_rainfall_max_6h',
     'HAZ_rainfall_max_24h',
     'HAZ_v_max',
     'HAZ_dis_track_min',
     'GEN_landslide_per',
     'GEN_stormsurge_per',
     'TOP_mean_slope',
     'TOP_mean_elevation_m',
     'TOP_ruggedness_stdev',
     'TOP_mean_ruggedness',
     'TOP_slope_stdev',
     'VUL_poverty_perc',
     'GEN_coast_length',
     'VUL_Housing_Units',
     'VUL_StrongRoof_StrongWall',
     'VUL_StrongRoof_SalvageWall',
     'VUL_LightRoof_StrongWall',
     'VUL_LightRoof_SalvageWall',
     'VUL_SalvagedRoof_StrongWall',
     'VUL_vulnerable_groups',
     'VUL_pantawid_pamilya_beneficiary'
]

#### Training the optimal model

In [23]:

file_name = "models/output/v1/selected_params_rf_binary2.p"
path = os.path.join(cdir, file_name)
pickle.dump(selected_params_rf_binary, open(path, "wb"))

file_name = "models/output/v1/df_predicted_rf_binary2.csv"
path = os.path.join(cdir, file_name)
df_predicted_rf_binary.to_csv(path, index=False)

C:\Users\ATEKLE~1\AppData\Local\Temp/ipykernel_18576/4103843247.py:3: ResourceWarning: unclosed file <_io.BufferedWriter name='C:\\Users\\ATeklesadik\\OneDrive - Rode Kruis\\Documents\\documents\\Typhoon-Impact-based-forecasting-model\\IBF-Typhoon-model\\models/output/v1/selected_params_rf_binary2.p'>
  pickle.dump(selected_params_rf_binary, open(path, "wb"))


In [39]:
# Setting the random forest search grid

 
rf_search_space = [
    {
        "rf__n_estimators": [500],
        "rf__max_depth": [22],
        "rf__min_samples_split": [2],
        "rf__min_samples_leaf": [3]
        
    }
]
# Obtaining the performance estimate
df_predicted_rf_binary, selected_params_rf_binary = rf_binary_performance(
    df_train_list=df_train_list,
    df_test_list=df_test_list,
    y_var='DAM_binary_dmg',
    features=selected_features_rf_binary,
    search_space=rf_search_space,
    stratK=True,
    cv_splits=5,
    class_weight="balanced",
    GS_score="f1",
    GS_randomized=False,
    GS_n_iter=50,
    verbose=10,
)

#n_samples / (n_classes * np.bincount(y))

Running for 1 out of a total of 40
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5; 1/1] START rf__max_depth=22, rf__min_samples_leaf=3, rf__min_samples_split=2, rf__n_estimators=500
[CV 1/5; 1/1] END rf__max_depth=22, rf__min_samples_leaf=3, rf__min_samples_split=2, rf__n_estimators=500;, score=(train=0.884, test=0.595) total time=  11.5s
[CV 2/5; 1/1] START rf__max_depth=22, rf__min_samples_leaf=3, rf__min_samples_split=2, rf__n_estimators=500
[CV 2/5; 1/1] END rf__max_depth=22, rf__min_samples_leaf=3, rf__min_samples_split=2, rf__n_estimators=500;, score=(train=0.884, test=0.603) total time=  12.2s
[CV 3/5; 1/1] START rf__max_depth=22, rf__min_samples_leaf=3, rf__min_samples_split=2, rf__n_estimators=500
[CV 3/5; 1/1] END rf__max_depth=22, rf__min_samples_leaf=3, rf__min_samples_split=2, rf__n_estimators=500;, score=(train=0.880, test=0.524) total time=  12.1s
[CV 4/5; 1/1] START rf__max_depth=22, rf__min_samples_leaf=3, rf__min_samples_split=2, rf__n_estimators=5

C:\Users\ATeklesadik\Miniconda3\envs\geo_env\lib\site-packages\sklearn\metrics\_classification.py:1495: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(


Selected Parameters: {'rf__max_depth': 22, 'rf__min_samples_leaf': 3, 'rf__min_samples_split': 2, 'rf__n_estimators': 500}
Train score: 0.8578856152512999
Test score: 0.0
Running for 3 out of a total of 40
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5; 1/1] START rf__max_depth=22, rf__min_samples_leaf=3, rf__min_samples_split=2, rf__n_estimators=500
[CV 1/5; 1/1] END rf__max_depth=22, rf__min_samples_leaf=3, rf__min_samples_split=2, rf__n_estimators=500;, score=(train=0.894, test=0.580) total time=  12.7s
[CV 2/5; 1/1] START rf__max_depth=22, rf__min_samples_leaf=3, rf__min_samples_split=2, rf__n_estimators=500
[CV 2/5; 1/1] END rf__max_depth=22, rf__min_samples_leaf=3, rf__min_samples_split=2, rf__n_estimators=500;, score=(train=0.903, test=0.518) total time=  12.2s
[CV 3/5; 1/1] START rf__max_depth=22, rf__min_samples_leaf=3, rf__min_samples_split=2, rf__n_estimators=500
[CV 3/5; 1/1] END rf__max_depth=22, rf__min_samples_leaf=3, rf__min_samples_split=2, rf__n_es

[CV 1/5; 1/1] END rf__max_depth=22, rf__min_samples_leaf=3, rf__min_samples_split=2, rf__n_estimators=500;, score=(train=0.882, test=0.577) total time=  12.8s
[CV 2/5; 1/1] START rf__max_depth=22, rf__min_samples_leaf=3, rf__min_samples_split=2, rf__n_estimators=500
[CV 2/5; 1/1] END rf__max_depth=22, rf__min_samples_leaf=3, rf__min_samples_split=2, rf__n_estimators=500;, score=(train=0.871, test=0.535) total time=  13.1s
[CV 3/5; 1/1] START rf__max_depth=22, rf__min_samples_leaf=3, rf__min_samples_split=2, rf__n_estimators=500
[CV 3/5; 1/1] END rf__max_depth=22, rf__min_samples_leaf=3, rf__min_samples_split=2, rf__n_estimators=500;, score=(train=0.871, test=0.564) total time=  12.7s
[CV 4/5; 1/1] START rf__max_depth=22, rf__min_samples_leaf=3, rf__min_samples_split=2, rf__n_estimators=500
[CV 4/5; 1/1] END rf__max_depth=22, rf__min_samples_leaf=3, rf__min_samples_split=2, rf__n_estimators=500;, score=(train=0.887, test=0.573) total time=  13.1s
[CV 5/5; 1/1] START rf__max_depth=22, rf

C:\Users\ATeklesadik\Miniconda3\envs\geo_env\lib\site-packages\sklearn\metrics\_classification.py:1495: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(


...

Selected Parameters: {'rf__max_depth': 22, 'rf__min_samples_leaf': 3, 'rf__min_samples_split': 2, 'rf__n_estimators': 500}
Train score: 0.8601216333622936
Test score: 0.0
Running for 35 out of a total of 40
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5; 1/1] START rf__max_depth=22, rf__min_samples_leaf=3, rf__min_samples_split=2, rf__n_estimators=500
[CV 1/5; 1/1] END rf__max_depth=22, rf__min_samples_leaf=3, rf__min_samples_split=2, rf__n_estimators=500;, score=(train=0.867, test=0.598) total time=  14.5s
[CV 2/5; 1/1] START rf__max_depth=22, rf__min_samples_leaf=3, rf__min_samples_split=2, rf__n_estimators=500
[CV 2/5; 1/1] END rf__max_depth=22, rf__min_samples_leaf=3, rf__min_samples_split=2, rf__n_estimators=500;, score=(train=0.871, test=0.558) total time=  14.1s
[CV 3/5; 1/1] START rf__max_depth=22, rf__min_samples_leaf=3, rf__min_samples_split=2, rf__n_estimators=500
[CV 3/5; 1/1] END rf__max_depth=22, rf__min_samples_leaf=3, rf__min_samples_split=2, rf__n_e

C:\Users\ATeklesadik\Miniconda3\envs\geo_env\lib\site-packages\sklearn\metrics\_classification.py:1495: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(


Selected Parameters: {'rf__max_depth': 22, 'rf__min_samples_leaf': 3, 'rf__min_samples_split': 2, 'rf__n_estimators': 500}
Train score: 0.8578856152512999
Test score: 0.0
Running for 36 out of a total of 40
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5; 1/1] START rf__max_depth=22, rf__min_samples_leaf=3, rf__min_samples_split=2, rf__n_estimators=500
[CV 1/5; 1/1] END rf__max_depth=22, rf__min_samples_leaf=3, rf__min_samples_split=2, rf__n_estimators=500;, score=(train=0.869, test=0.612) total time=  13.2s
[CV 2/5; 1/1] START rf__max_depth=22, rf__min_samples_leaf=3, rf__min_samples_split=2, rf__n_estimators=500
[CV 2/5; 1/1] END rf__max_depth=22, rf__min_samples_leaf=3, rf__min_samples_split=2, rf__n_estimators=500;, score=(train=0.877, test=0.554) total time=  13.1s
[CV 3/5; 1/1] START rf__max_depth=22, rf__min_samples_leaf=3, rf__min_samples_split=2, rf__n_estimators=500
[CV 3/5; 1/1] END rf__max_depth=22, rf__min_samples_leaf=3, rf__min_samples_split=2, rf__n_e

C:\Users\ATeklesadik\Miniconda3\envs\geo_env\lib\site-packages\sklearn\metrics\_classification.py:1495: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(


Selected Parameters: {'rf__max_depth': 22, 'rf__min_samples_leaf': 3, 'rf__min_samples_split': 2, 'rf__n_estimators': 500}
Train score: 0.8512467755803954
Test score: 0.0
Running for 37 out of a total of 40
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5; 1/1] START rf__max_depth=22, rf__min_samples_leaf=3, rf__min_samples_split=2, rf__n_estimators=500
[CV 1/5; 1/1] END rf__max_depth=22, rf__min_samples_leaf=3, rf__min_samples_split=2, rf__n_estimators=500;, score=(train=0.877, test=0.510) total time=  12.0s
[CV 2/5; 1/1] START rf__max_depth=22, rf__min_samples_leaf=3, rf__min_samples_split=2, rf__n_estimators=500
[CV 2/5; 1/1] END rf__max_depth=22, rf__min_samples_leaf=3, rf__min_samples_split=2, rf__n_estimators=500;, score=(train=0.881, test=0.602) total time=  12.9s
[CV 3/5; 1/1] START rf__max_depth=22, rf__min_samples_leaf=3, rf__min_samples_split=2, rf__n_estimators=500
[CV 3/5; 1/1] END rf__max_depth=22, rf__min_samples_leaf=3, rf__min_samples_split=2, rf__n_e

#### XG Boost

In [47]:

combined_input_data = combined_input_data[combined_input_data['DAM_binary_dmg'].notnull()]
X = combined_input_data[features]
y = combined_input_data["DAM_binary_dmg"]

# Setting the train and the test sets for obtaining performance estimate
df_train_list, df_test_list = splitting_train_test(combined_input_data)

In [25]:
# Setting the XGBoost search grid for full dataset
xgb_search_space = [
    {
        "estimator__learning_rate": [0.1, 0.5, 1],
        "estimator__gamma": [0.1, 0.5, 2],#0
        "estimator__max_depth": [6, 8],
        "estimator__reg_lambda": [0.001, 0.1, 1],
        "estimator__n_estimators": [100, 200],
        "estimator__colsample_bytree": [0.5, 0.7],
    }
]

# Obtaining the selected features based on the full dataset
selected_features_xgb_binary, selected_params_xgb_binary_full = xgb_binary_features(
    X=X,
    y=y,
    features=features,
    search_space=xgb_search_space,
    objective="binary:hinge",
    cv_splits=5,
    min_features_to_select=1,
    GS_score="f1",
    GS_n_iter=50,
    GS_randomized=True,
    verbose=10,
)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV 1/5; 1/50] START estimator__colsample_bytree=0.7, estimator__gamma=2, estimator__learning_rate=0.5, estimator__max_depth=6, estimator__n_estimators=200, estimator__reg_lambda=0.1
[21:19:06] WARNING: D:\bld\xgboost-split_1631820768043\work\src\learner.cc:573: 
Parameters: { "scale_pos_weight" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




C:\Users\ATeklesadik\Miniconda3\envs\geo_env\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


[21:19:08] WARNING: D:\bld\xgboost-split_1631820768043\work\src\learner.cc:573: 
Parameters: { "scale_pos_weight" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




C:\Users\ATeklesadik\Miniconda3\envs\geo_env\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


[21:19:09] WARNING: D:\bld\xgboost-split_1631820768043\work\src\learner.cc:573: 
Parameters: { "scale_pos_weight" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




C:\Users\ATeklesadik\Miniconda3\envs\geo_env\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


[21:19:10] WARNING: D:\bld\xgboost-split_1631820768043\work\src\learner.cc:573: 
Parameters: { "scale_pos_weight" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




C:\Users\ATeklesadik\Miniconda3\envs\geo_env\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


[21:19:12] WARNING: D:\bld\xgboost-split_1631820768043\work\src\learner.cc:573: 
Parameters: { "scale_pos_weight" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




KeyboardInterrupt: 

#### Training the optimal model

In [48]:
selected_features_xgb_regr =[
    'HAZ_v_max',
    'HAZ_dis_track_min',
    'VUL_StrongRoof_StrongWall',
    'TOP_mean_elevation_m',
    'HAZ_rainfall_max_6h',
    'HAZ_rainfall_max_24h',
    'HAZ_rainfall_Total',
    'VUL_vulnerable_groups',
    'VUL_pantawid_pamilya_beneficiary',
    'VUL_StrongRoof_LightWall',
    'VUL_poverty_perc',
    'TOP_ruggedness_stdev',
    'TOP_slope_stdev',
    'TOP_mean_slope',
    'GEN_coast_length',
    'VUL_Housing_Units',
    'GEN_stormsurge_per',
    'GEN_landslide_per',
    'TOP_mean_ruggedness',
    'GEN_Yel_per_LSSAb',
    'GEN_Yellow_per_LSbl',
    'GEN_Red_per_LSbldg',
    'GEN_with_coast',
    'GEN_OR_per_SSAbldg',
    'GEN_RED_per_SSAbldg',
    'GEN_Or_per_LSblg']

In [49]:

df_train_list, df_test_list = splitting_train_test(combined_input_data)

selected_features_xgb_binary=selected_features_xgb_regr


xgb_search_space = [
    {
        "xgb__learning_rate": [0.3], #0.03
        "xgb__gamma": [0.1], #0
        "xgb__max_depth": [6], #6
        "xgb__reg_lambda": [0.001],
        "xgb__n_estimators": [50],
        "xgb__colsample_bytree": [0.7],#1
    }
]
# Obtaining the performance estimate
df_predicted_xgb_binary, selected_params_xgb_binary = xgb_binary_performance(
    df_train_list=df_train_list,
    df_test_list=df_test_list,
    y_var='DAM_binary_dmg',
    features=selected_features_xgb_binary,
    search_space=xgb_search_space,
    stratK=True,
    cv_splits=5,
    objective="binary:hinge",
    GS_score="f1",
    GS_randomized=True,
    GS_n_iter=100,
    verbose=10,
)

file_name = "models/output/v1/selected_params_xgb_binary.p"
path = os.path.join(cdir, file_name)
pickle.dump(selected_params_xgb_binary, open(path, "wb"))

file_name = "models/output/v1/df_predicted_xgb_binary.csv"
path = os.path.join(cdir, file_name)
df_predicted_xgb_binary.to_csv(path, index=False)

Running for 1 out of a total of 40
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5; 1/1] START xgb__colsample_bytree=0.7, xgb__gamma=0.1, xgb__learning_rate=0.3, xgb__max_depth=6, xgb__n_estimators=50, xgb__reg_lambda=0.001
[18:08:59] WARNING: D:\bld\xgboost-split_1631820768043\work\src\learner.cc:573: 
Parameters: { "scale_pos_weight" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




C:\Users\ATeklesadik\Miniconda3\envs\geo_env\lib\site-packages\sklearn\model_selection\_search.py:285: UserWarning: The total space of parameters 1 is smaller than n_iter=100. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
C:\Users\ATeklesadik\Miniconda3\envs\geo_env\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
C:\Users\ATeklesadik\Miniconda3\envs\geo_env\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


[CV 1/5; 1/1] END xgb__colsample_bytree=0.7, xgb__gamma=0.1, xgb__learning_rate=0.3, xgb__max_depth=6, xgb__n_estimators=50, xgb__reg_lambda=0.001;, score=(train=0.983, test=0.473) total time=   0.5s
[CV 2/5; 1/1] START xgb__colsample_bytree=0.7, xgb__gamma=0.1, xgb__learning_rate=0.3, xgb__max_depth=6, xgb__n_estimators=50, xgb__reg_lambda=0.001
[18:08:59] WARNING: D:\bld\xgboost-split_1631820768043\work\src\learner.cc:573: 
Parameters: { "scale_pos_weight" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




C:\Users\ATeklesadik\Miniconda3\envs\geo_env\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
C:\Users\ATeklesadik\Miniconda3\envs\geo_env\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


[CV 2/5; 1/1] END xgb__colsample_bytree=0.7, xgb__gamma=0.1, xgb__learning_rate=0.3, xgb__max_depth=6, xgb__n_estimators=50, xgb__reg_lambda=0.001;, score=(train=0.995, test=0.520) total time=   0.6s
[CV 3/5; 1/1] START xgb__colsample_bytree=0.7, xgb__gamma=0.1, xgb__learning_rate=0.3, xgb__max_depth=6, xgb__n_estimators=50, xgb__reg_lambda=0.001
[18:09:00] WARNING: D:\bld\xgboost-split_1631820768043\work\src\learner.cc:573: 
Parameters: { "scale_pos_weight" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




...

C:\Users\ATeklesadik\Miniconda3\envs\geo_env\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
C:\Users\ATeklesadik\Miniconda3\envs\geo_env\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


[CV 4/5; 1/1] END xgb__colsample_bytree=0.7, xgb__gamma=0.1, xgb__learning_rate=0.3, xgb__max_depth=6, xgb__n_estimators=50, xgb__reg_lambda=0.001;, score=(train=0.970, test=0.565) total time=   0.7s
[CV 5/5; 1/1] START xgb__colsample_bytree=0.7, xgb__gamma=0.1, xgb__learning_rate=0.3, xgb__max_depth=6, xgb__n_estimators=50, xgb__reg_lambda=0.001
[18:12:22] WARNING: D:\bld\xgboost-split_1631820768043\work\src\learner.cc:573: 
Parameters: { "scale_pos_weight" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




C:\Users\ATeklesadik\Miniconda3\envs\geo_env\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
C:\Users\ATeklesadik\Miniconda3\envs\geo_env\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


[CV 5/5; 1/1] END xgb__colsample_bytree=0.7, xgb__gamma=0.1, xgb__learning_rate=0.3, xgb__max_depth=6, xgb__n_estimators=50, xgb__reg_lambda=0.001;, score=(train=0.987, test=0.554) total time=   0.7s
[18:12:23] WARNING: D:\bld\xgboost-split_1631820768043\work\src\learner.cc:573: 
Parameters: { "scale_pos_weight" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Selected Parameters: {'xgb__reg_lambda': 0.001, 'xgb__n_estimators': 50, 'xgb__max_depth': 6, 'xgb__learning_rate': 0.3, 'xgb__gamma': 0.1, 'xgb__colsample_bytree': 0.7}
Train score: 0.9636552440290759
Test score: 0.0


C:\Users\ATeklesadik\Miniconda3\envs\geo_env\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
C:\Users\ATeklesadik\Miniconda3\envs\geo_env\lib\site-packages\sklearn\metrics\_classification.py:1495: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
C:\Users\ATEKLE~1\AppData\Local\Temp/ipykernel_18576/2302247418.py:34: ResourceWarning: unclosed file <_io.BufferedWriter name='C:\\Users\\ATeklesadik\\OneDrive - Rode Kruis\\Documents\\documents\\Typhoon-Impact-based-forecasting-model\\IBF-Typhoon-model\\models/output/v1/selected_params_xgb_binary.p'>
  pickle.dump(selected_params_xgb_binary, open(path, "wb"))


#### Base line

In [29]:

def unweighted_random(y_train, y_test):
    options = y_train.value_counts(normalize=True)
    y_pred = random.choices(population=list(options.index), k=len(y_test))
    return y_pred

def weighted_random(y_train, y_test):
    options = y_train.value_counts()
    y_pred = random.choices(
        population=list(options.index), weights=list(options.values), k=len(y_test)
    )
    return y_pred

df_predicted_random = pd.DataFrame(columns=["typhoon", "actual", "predicted"])

for i in range(len(df_train_list)):

    train = df_train_list[i]
    test = df_test_list[i]

    y_train = train["DAM_binary_dmg"]
    y_test = test["DAM_binary_dmg"]

    y_pred_test = unweighted_random(y_train, y_test)
    df_predicted_temp = pd.DataFrame(
        {"typhoon": test["typhoon"], "actual": y_test, "predicted": y_pred_test}
    )

    df_predicted_random = pd.concat([df_predicted_random, df_predicted_temp])


file_name = "models/output/v1/df_predicted_random.csv"
path = os.path.join(cdir, file_name)
df_predicted_random.to_csv(path, index=False)
    
df_predicted_random_weighted = pd.DataFrame(columns=["typhoon", "actual", "predicted"])

for i in range(len(df_train_list)):

    train = df_train_list[i]
    test = df_test_list[i]

    y_train = train["DAM_binary_dmg"]
    y_test = test["DAM_binary_dmg"]

    y_pred_test = weighted_random(y_train, y_test)
    df_predicted_temp = pd.DataFrame(
        {"typhoon": test["typhoon"], "actual": y_test, "predicted": y_pred_test}
    )

    df_predicted_random_weighted = pd.concat(
        [df_predicted_random_weighted, df_predicted_temp]
    )

    
file_name = "models/output/v1/df_predicted_random_weighted.csv"
path = os.path.join(cdir, file_name)
df_predicted_random_weighted.to_csv(path, index=False)
 